In [1]:
import os
import pandas as pd

# Path to the folder containing the pickle files
folder_path = "../data/raw/transaction/"

# Define the start and end dates
start_date = "2025-01-01"
end_date = "2025-01-01"

# List all pickle files in the folder
pickle_files = [f for f in os.listdir(folder_path) if f.endswith(".pkl")]

# Filter files within the date range
filtered_files = [
    f for f in pickle_files
    if start_date <= f.split(".pkl")[0] <= end_date
]

# Initialize an empty list to store DataFrames
dataframes = []

# Iterate through each filtered pickle file and load it into a DataFrame
for pickle_file in filtered_files:
    file_path = os.path.join(folder_path, pickle_file)
    df = pd.read_pickle(file_path)
    dataframes.append(df)

# Concatenate all DataFrames into one
transactions_df = pd.concat(dataframes, ignore_index=True)

# Display the first few rows of the final DataFrame
print(transactions_df.head())


   TRANSACTION_ID         TX_DATETIME CUSTOMER_ID TERMINAL_ID  TX_AMOUNT  \
0         2051331 2025-01-01 00:01:33        1959        9559     143.11   
1         2051332 2025-01-01 00:02:26         658        9395      73.66   
2         2051333 2025-01-01 00:03:36        4371        7721      99.55   
3         2051334 2025-01-01 00:04:03         802        6022      50.71   
4         2051335 2025-01-01 00:04:49        4630        2924      23.60   

  TX_TIME_SECONDS TX_TIME_DAYS  TX_FRAUD  TX_FRAUD_SCENARIO  
0        18489693          214         0                  0  
1        18489746          214         0                  0  
2        18489816          214         0                  0  
3        18489843          214         0                  0  
4        18489889          214         0                  0  


In [2]:
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, StructType, StructField

In [4]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
                .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.7.1,org.apache.hadoop:hadoop-aws:3.3.2")
                .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000") 
                .config("spark.hadoop.fs.s3a.access.key", "minio") 
                .config("spark.hadoop.fs.s3a.secret.key", "minio123") 
                .config("spark.hadoop.fs.s3a.path.style.access", "true") 
                .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
                .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
                .config("spark.sql.catalog.spark_catalog.type", "hadoop")
                .config("spark.sql.catalog.spark_catalog.warehouse", "s3a://commerce/warehouse/") 
                .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") 
                .appName("data_store")
         .getOrCreate())

25/02/13 21:29:08 WARN Utils: Your hostname, Gauravs-MacBook-Air-3.local resolves to a loopback address: 127.0.0.1; using 192.168.1.6 instead (on interface en0)
25/02/13 21:29:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/saurabh/Desktop/workspace/realtime_fraud_detection_system/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/saurabh/.ivy2/cache
The jars for the packages stored in: /Users/saurabh/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7ada9a4b-4833-4a61-9f10-a94854880f14;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in ce

In [5]:
transactions_df=transactions_df[["TRANSACTION_ID","TX_DATETIME","CUSTOMER_ID","TERMINAL_ID","TX_AMOUNT"]]

In [6]:
transactions_sdf=spark.createDataFrame(transactions_df)

In [7]:
t1=transactions_sdf.withColumnRenamed("TRANSACTION_ID","tx_id") \
.withColumnRenamed("TX_DATETIME","tx_datetime") \
.withColumnRenamed("CUSTOMER_ID","customer_id") \
.withColumnRenamed("TERMINAL_ID","terminal_id") \
.withColumnRenamed("TX_AMOUNT","tx_amount")

In [8]:
from pyspark.sql.functions import current_timestamp
transactions_sdf1=t1.withColumn("row_created_timestamp", current_timestamp()) \
.withColumn("row_updated_timestamp", current_timestamp())

In [9]:
transactions_sdf1.createOrReplaceTempView("transaction1")

25/02/13 21:30:18 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [10]:
spark.sql("select * from transaction1 ").show()

+-------+-------------------+-----------+-----------+---------+---------------------+---------------------+
|  tx_id|        tx_datetime|customer_id|terminal_id|tx_amount|row_created_timestamp|row_updated_timestamp|
+-------+-------------------+-----------+-----------+---------+---------------------+---------------------+
|2051331|2025-01-01 00:01:33|       1959|       9559|   143.11| 2025-02-13 21:30:...| 2025-02-13 21:30:...|
|2051332|2025-01-01 00:02:26|        658|       9395|    73.66| 2025-02-13 21:30:...| 2025-02-13 21:30:...|
|2051333|2025-01-01 00:03:36|       4371|       7721|    99.55| 2025-02-13 21:30:...| 2025-02-13 21:30:...|
|2051334|2025-01-01 00:04:03|        802|       6022|    50.71| 2025-02-13 21:30:...| 2025-02-13 21:30:...|
|2051335|2025-01-01 00:04:49|       4630|       2924|     23.6| 2025-02-13 21:30:...| 2025-02-13 21:30:...|
|2051336|2025-01-01 00:05:58|       1837|       1216|    56.17| 2025-02-13 21:30:...| 2025-02-13 21:30:...|
|2051337|2025-01-01 00:06:26

In [11]:
spark.sql("""
      CREATE TABLE IF NOT EXISTS payment.transactions (
                    tx_id INT,
                    tx_datetime Timestamp,
                    customer_id INT,
                    terminal_id INT,
                    tx_amount DECIMAL(10,2),
    row_created_timestamp TIMESTAMP,
    row_updated_timestamp TIMESTAMP
)
USING iceberg
""")

DataFrame[]

In [12]:
spark.sql("""
    MERGE INTO payment.transactions AS target
    USING transaction1 AS source
ON target.tx_id = source.tx_id
WHEN MATCHED THEN
  UPDATE SET
    target.tx_datetime = source.tx_datetime,
    target.customer_id = source.customer_id,
    target.terminal_id = source.terminal_id,
    target.tx_amount = source.tx_amount,      
    target.row_updated_timestamp = source.row_updated_timestamp
WHEN NOT MATCHED THEN
  INSERT (          tx_id,
                    tx_datetime,
                    customer_id,
                    terminal_id,
                    tx_amount,
    row_created_timestamp,
    row_updated_timestamp
  )
  VALUES (
    source.tx_id,
    source.tx_datetime,
    source.customer_id,
    source.terminal_id,
    source.tx_amount,
    source.row_created_timestamp,
    source.row_updated_timestamp
  )
""")

DataFrame[]